pip install transformers[torch] datasets evaluate opendatasets

In [5]:
import os
import mlflow
import datasets
import transformers
import evaluate
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer
import opendatasets as od
import pandas as pd
import numpy as np
import pickle as pkl

In [2]:
od.download(
    "https://www.kaggle.com/datasets/rmisra/politifact-fact-check-dataset/data"
    )
data = datasets.Dataset.from_json("politifact-fact-check-dataset/politifact_factcheck_data.json")
data = data.train_test_split(train_size=0.8)

Using custom data configuration default-58a22729889bbb0b
Found cached dataset json (C:/Users/Abelda-san/.cache/huggingface/datasets/json/default-58a22729889bbb0b/0.0.0)


Skipping, found downloaded files in ".\politifact-fact-check-dataset" (use force=True to force download)


In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

label2id = {
    'true': 0,
    'mostly-true': 1,
    'half-true': 2,
    'mostly-false': 3,
    'false': 4,
    'pants-fire': 5
}
id2label = {
    0: 'true',
    1: 'mostly-true',
    2: 'half-true',
    3: 'mostly-false',
    4: 'false',
    5: 'pants-fire'
}
num_labels = len(label2id)

def preprocess_data(examples):

    text = examples["statement"]
    labels = pd.Series(examples['verdict'])

    encoding = tokenizer(text, padding=True, truncation=True)
    encoding['labels'] = list(labels.apply(lambda x: [1.0 if label2id[x]==i else 0.0 for i in range(num_labels)]))

    return encoding


train_tokenized_dataset = data["train"].map(preprocess_data, batched=True, remove_columns=data["train"].column_names)
test_tokenized_dataset = data["test"].map(preprocess_data, batched=True, remove_columns=data["test"].column_names)

train_tokenized_dataset.set_format("torch")
test_tokenized_dataset.set_format("torch")

 80%|████████  | 4/5 [00:00<00:00,  8.36ba/s]


In [6]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                            problem_type="multi_label_classification", 
                                                            num_labels=6,
                                                            id2label=id2label,
                                                            label2id=label2id)

Downloading: 100%|██████████| 268M/268M [02:48<00:00, 1.59MB/s] 
c:\Users\Abelda-san\anaconda3\envs\factcheck\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Abelda-san\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at distilbert-base-unc

In [ ]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    compute_metrics=compute_metrics,

)

In [ ]:
trainer.train()